# Single-image Super-resolution Generative Adversarial Networks - Training

In [1]:
from __future__ import print_function, division
import numpy as np
import scipy
import os
import sys
import glob
from skimage.transform import resize
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras.layers import Input
import matplotlib.pylab as plt
import datetime
MY_UTILS_PATH = "../Modules/"
if not MY_UTILS_PATH in sys.path:
    sys.path.append(MY_UTILS_PATH)
import sr_gan
from random import shuffle

/home/roberto/Documents/virtualenv/venv04/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def imread(path):
    return scipy.misc.imread(path, mode = "RGB").astype(np.float)

def load_data(data_list,img_res=(128, 128), batch_size = 1):
    batch_images = np.random.choice(data_list, size=batch_size)
    imgs_hr = []
    imgs_lr = []
    for img_path in batch_images:
        img = imread(img_path)
        img_lr = resize(img, img_res,anti_aliasing = True)
        # If training => do random flip
        if np.random.random() < 0.5:
            img = np.fliplr(img)
            img_lr = np.fliplr(img_lr)

        imgs_hr.append(img)
        imgs_lr.append(img_lr)
    imgs_hr = np.array(imgs_hr) / 127.5 - 1.
    imgs_lr = np.array(imgs_lr) / 127.5 - 1.
    return imgs_hr, imgs_lr

def save_imgs(epoch,val,batch_size,generator):
    r, c = 2, 2

    imgs_hr, imgs_lr = load_data(val,img_res=(128, 128), batch_size = batch_size)
    fake_hr = generator.predict(imgs_lr)

    # Rescale images 0 - 1
    imgs_lr = 0.5 * imgs_lr + 0.5
    fake_hr = 0.5 * fake_hr + 0.5
    imgs_hr = 0.5 * imgs_hr + 0.5

    # Save generated images and the high resolution originals
    titles = ['Generated', 'Original']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for row in range(r):
        for col, image in enumerate([fake_hr, imgs_hr]):
            axs[row, col].imshow(image[row])
            axs[row, col].set_title(titles[col])
            axs[row, col].axis('off')
        cnt += 1
    fig.savefig("../Data/gan_images/%d.png" % (epoch))
    plt.close()

    # Save low resolution images for comparison
    for i in range(r):
        fig = plt.figure()
        plt.imshow(imgs_lr[i])
        fig.savefig('../Data/gan_images/%d_lowres%d.png' % (epoch, i))
        plt.close()

In [3]:
g_loss = 1.0e+20

# Input shape
channels = 3
lr_height = 128                 # Low resolution height
lr_width = 128                  # Low resolution width
lr_shape = (lr_height, lr_width, channels)
hr_height = lr_height*2   # High resolution height
hr_width = lr_width*2     # High resolution width
hr_shape = (hr_height, hr_width, channels)
df = 64
optimizer = Adam(0.0002, 0.5)

vgg = sr_gan.build_vgg(hr_shape)
vgg.trainable = False
vgg.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

# Calculate output shape of D (PatchGAN)
patch = int(hr_height / 2**4)
disc_patch = (patch, patch, 1)

# Build and compile the discriminator
discriminator = sr_gan.build_discriminator(hr_shape,df)
discriminator.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

# Build and compile the generator
generator = sr_gan.build_generator(nchannels= channels)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# High res. and low res. images
img_hr = Input(shape=hr_shape)
img_lr = Input(shape=lr_shape)

# Generate high res. version from low res.
fake_hr = generator(img_lr)
# Extract image features of the generated img
fake_features = vgg(fake_hr)

# For the combined model we will only train the generator
discriminator.trainable = False

# Discriminator determines validity of generated high res. images
validity = discriminator(fake_hr)

# Combined model
combined = Model([img_lr, img_hr], [validity, fake_features])
combined.compile(loss=['binary_crossentropy', 'mse'],
                      loss_weights=[1e-3, 1],
                      optimizer=optimizer)

In [4]:
epochs = 15000
batch_size=4
save_interval=102
start_time = datetime.datetime.now()

path = glob.glob('../Data/carotid_slices/*.png')
shuffle(path)
discriminator.load_weights("../Data/discriminator.hdf5")
generator.load_weights("../Data/generator.hdf5")
combined.load_weights("../Data/combined.hdf5") 

for epoch in xrange(epochs):

    # ----------------------
    #  Train Discriminator
    # ----------------------

    # Sample images and their conditioning counterparts
    imgs_hr, imgs_lr = load_data(path,img_res=(128, 128), batch_size = batch_size)

    # From low res. image generate high res. version
    fake_hr = generator.predict(imgs_lr)

    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    # Train the discriminators (original images = real / generated = Fake)
    d_loss_real = discriminator.train_on_batch(imgs_hr, valid)
    d_loss_fake = discriminator.train_on_batch(fake_hr, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ------------------
    #  Train Generator
    # ------------------

    # Sample images and their conditioning counterparts
    imgs_hr, imgs_lr = load_data(path,img_res=(128, 128), batch_size = batch_size)

    # The generators want the discriminators to label the generated images as real
    valid = np.ones((batch_size,) + disc_patch)

    # Extract ground truth image features using pre-trained VGG19 model
    image_features = vgg.predict(imgs_hr)

    # Train the generators
    g_loss_new = combined.train_on_batch([imgs_lr, imgs_hr], [valid, image_features])
    if g_loss_new[0] < g_loss:
        g_loss = g_loss_new[0]   
        discriminator.save_weights("../Data/discriminator.hdf5")
        generator.save_weights("../Data/generator.hdf5")
        combined.save_weights("../Data/combined.hdf5") 

    elapsed_time = datetime.datetime.now() - start_time
    # Plot the progress
    print ("%d time: %s" % (epoch, elapsed_time))
    print ("Loss: %f" %g_loss) 
    # If at save interval => save generated image samples
    if epoch % save_interval == 0:
        save_imgs(epoch,path,batch_size,generator)

/home/roberto/Documents/virtualenv/venv04/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/roberto/Documents/virtualenv/venv04/local/lib/python2.7/site-packages/scikit_image-0.14.dev0-py2.7-linux-x86_64.egg/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/roberto/Documents/virtualenv/venv04/local/lib/python2.7/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 time: 0:00:36.698645
Loss: 0.230009
1 time: 0:00:38.473552
Loss: 0.230009
2 time: 0:00:39.716861
Loss: 0.230009
3 time: 0:00:40.944455
Loss: 0.230009
4 time: 0:00:42.219147
Loss: 0.230009
5 time: 0:00:43.431128
Loss: 0.230009
6 time: 0:00:44.739399
Loss: 0.230009
7 time: 0:00:46.023398
Loss: 0.230009
8 time: 0:00:47.272561
Loss: 0.230009
9 time: 0:00:48.494109
Loss: 0.230009
10 time: 0:00:49.760620
Loss: 0.230009
11 time: 0:00:51.061886
Loss: 0.230009
12 time: 0:00:52.405251
Loss: 0.230009
13 time: 0:00:53.703739
Loss: 0.230009
14 time: 0:00:55.035664
Loss: 0.230009
15 time: 0:00:56.275028
Loss: 0.230009
16 time: 0:00:57.558858
Loss: 0.230009
17 time: 0:00:58.831468
Loss: 0.230009
18 time: 0:01:04.354518
Loss: 0.229186
19 time: 0:01:06.025823
Loss: 0.229186
20 time: 0:01:07.698704
Loss: 0.229186
21 time: 0:01:09.320542
Loss: 0.229186
22 time: 0:01:10.985420
Loss: 0.229186
23 time: 0:01:12.758962
Loss: 0.229186
24 time: 0:01:14.543237
Loss: 0.229186
25 time: 0:01:20.056322
Loss: 0.228

208 time: 0:05:43.486357
Loss: 0.179367
209 time: 0:05:44.835369
Loss: 0.179367
210 time: 0:05:46.210340
Loss: 0.179367
211 time: 0:05:47.547559
Loss: 0.179367
212 time: 0:05:48.876394
Loss: 0.179367
213 time: 0:05:50.277366
Loss: 0.179367
214 time: 0:05:51.637707
Loss: 0.179367
215 time: 0:05:53.035065
Loss: 0.179367
216 time: 0:05:54.348538
Loss: 0.179367
217 time: 0:05:55.750194
Loss: 0.179367
218 time: 0:05:57.099661
Loss: 0.179367
219 time: 0:05:58.386269
Loss: 0.179367
220 time: 0:05:59.708130
Loss: 0.179367
221 time: 0:06:01.039747
Loss: 0.179367
222 time: 0:06:02.476684
Loss: 0.179367
223 time: 0:06:03.776170
Loss: 0.179367
224 time: 0:06:05.093100
Loss: 0.179367
225 time: 0:06:06.452356
Loss: 0.179367
226 time: 0:06:07.718181
Loss: 0.179367
227 time: 0:06:09.004713
Loss: 0.179367
228 time: 0:06:10.366981
Loss: 0.179367
229 time: 0:06:11.798718
Loss: 0.179367
230 time: 0:06:13.185354
Loss: 0.179367
231 time: 0:06:14.532031
Loss: 0.179367
232 time: 0:06:15.940893
Loss: 0.179367


413 time: 0:10:14.659062
Loss: 0.179367
414 time: 0:10:15.941072
Loss: 0.179367
415 time: 0:10:17.141440
Loss: 0.179367
416 time: 0:10:19.092138
Loss: 0.179367
417 time: 0:10:20.383837
Loss: 0.179367
418 time: 0:10:21.630956
Loss: 0.179367
419 time: 0:10:22.884929
Loss: 0.179367
420 time: 0:10:24.220861
Loss: 0.179367
421 time: 0:10:25.496369
Loss: 0.179367
422 time: 0:10:26.736260
Loss: 0.179367
423 time: 0:10:27.998613
Loss: 0.179367
424 time: 0:10:29.772300
Loss: 0.179367
425 time: 0:10:30.980567
Loss: 0.179367
426 time: 0:10:32.246721
Loss: 0.179367
427 time: 0:10:33.439028
Loss: 0.179367
428 time: 0:10:34.711320
Loss: 0.179367
429 time: 0:10:36.048733
Loss: 0.179367
430 time: 0:10:37.413147
Loss: 0.179367
431 time: 0:10:38.762908
Loss: 0.179367
432 time: 0:10:40.052598
Loss: 0.179367
433 time: 0:10:41.418792
Loss: 0.179367
434 time: 0:10:42.730923
Loss: 0.179367
435 time: 0:10:49.766982
Loss: 0.155233
436 time: 0:10:51.038042
Loss: 0.155233
437 time: 0:10:52.324047
Loss: 0.155233


618 time: 0:14:54.487369
Loss: 0.155233
619 time: 0:14:55.700890
Loss: 0.155233
620 time: 0:14:57.069905
Loss: 0.155233
621 time: 0:14:58.275015
Loss: 0.155233
622 time: 0:14:59.535733
Loss: 0.155233
623 time: 0:15:01.677977
Loss: 0.155233
624 time: 0:15:02.973554
Loss: 0.155233
625 time: 0:15:04.230986
Loss: 0.155233
626 time: 0:15:05.510741
Loss: 0.155233
627 time: 0:15:06.847580
Loss: 0.155233
628 time: 0:15:08.009795
Loss: 0.155233
629 time: 0:15:09.260197
Loss: 0.155233
630 time: 0:15:10.678205
Loss: 0.155233
631 time: 0:15:12.774240
Loss: 0.155233
632 time: 0:15:14.025546
Loss: 0.155233
633 time: 0:15:15.378957
Loss: 0.155233
634 time: 0:15:16.722838
Loss: 0.155233
635 time: 0:15:18.032250
Loss: 0.155233
636 time: 0:15:19.286913
Loss: 0.155233
637 time: 0:15:20.527519
Loss: 0.155233
638 time: 0:15:21.791732
Loss: 0.155233
639 time: 0:15:23.036609
Loss: 0.155233
640 time: 0:15:24.295673
Loss: 0.155233
641 time: 0:15:25.464608
Loss: 0.155233
642 time: 0:15:26.821179
Loss: 0.155233


...

14144 time: 4:50:54.555409
Loss: 0.121898
14145 time: 4:50:55.770142
Loss: 0.121898
14146 time: 4:50:56.949665
Loss: 0.121898
14147 time: 4:50:58.138594
Loss: 0.121898
14148 time: 4:50:59.292541
Loss: 0.121898
14149 time: 4:51:00.502549
Loss: 0.121898
14150 time: 4:51:01.669182
Loss: 0.121898
14151 time: 4:51:02.806401
Loss: 0.121898
14152 time: 4:51:03.963780
Loss: 0.121898
14153 time: 4:51:05.153378
Loss: 0.121898
14154 time: 4:51:06.287325
Loss: 0.121898
14155 time: 4:51:07.450721
Loss: 0.121898
14156 time: 4:51:08.589596
Loss: 0.121898
14157 time: 4:51:09.895045
Loss: 0.121898
14158 time: 4:51:11.098345
Loss: 0.121898
14159 time: 4:51:12.333014
Loss: 0.121898
14160 time: 4:51:13.507403
Loss: 0.121898
14161 time: 4:51:14.661463
Loss: 0.121898
14162 time: 4:51:15.794173
Loss: 0.121898
14163 time: 4:51:16.969500
Loss: 0.121898
14164 time: 4:51:18.084508
Loss: 0.121898
14165 time: 4:51:19.240922
Loss: 0.121898
14166 time: 4:51:20.349071
Loss: 0.121898
14167 time: 4:51:21.466194
Loss: 0

14340 time: 4:54:46.209638
Loss: 0.121898
14341 time: 4:54:47.361911
Loss: 0.121898
14342 time: 4:54:48.585347
Loss: 0.121898
14343 time: 4:54:49.727046
Loss: 0.121898
14344 time: 4:54:50.816258
Loss: 0.121898
14345 time: 4:54:51.958149
Loss: 0.121898
14346 time: 4:54:53.096462
Loss: 0.121898
14347 time: 4:54:54.256011
Loss: 0.121898
14348 time: 4:54:55.332882
Loss: 0.121898
14349 time: 4:54:56.426982
Loss: 0.121898
14350 time: 4:54:57.543870
Loss: 0.121898
14351 time: 4:54:58.663687
Loss: 0.121898
14352 time: 4:54:59.837946
Loss: 0.121898
14353 time: 4:55:00.957813
Loss: 0.121898
14354 time: 4:55:02.059892
Loss: 0.121898
14355 time: 4:55:03.221202
Loss: 0.121898
14356 time: 4:55:04.689338
Loss: 0.121898
14357 time: 4:55:06.107076
Loss: 0.121898
14358 time: 4:55:07.481225
Loss: 0.121898
14359 time: 4:55:08.705247
Loss: 0.121898
14360 time: 4:55:09.954958
Loss: 0.121898
14361 time: 4:55:11.209151
Loss: 0.121898
14362 time: 4:55:12.450446
Loss: 0.121898
14363 time: 4:55:13.718504
Loss: 0

14536 time: 4:58:41.251230
Loss: 0.121898
14537 time: 4:58:42.444033
Loss: 0.121898
14538 time: 4:58:43.595705
Loss: 0.121898
14539 time: 4:58:44.770805
Loss: 0.121898
14540 time: 4:58:45.908877
Loss: 0.121898
14541 time: 4:58:47.086179
Loss: 0.121898
14542 time: 4:58:48.227976
Loss: 0.121898
14543 time: 4:58:49.368887
Loss: 0.121898
14544 time: 4:58:50.459668
Loss: 0.121898
14545 time: 4:58:51.597228
Loss: 0.121898
14546 time: 4:58:52.681795
Loss: 0.121898
14547 time: 4:58:53.842055
Loss: 0.121898
14548 time: 4:58:54.969685
Loss: 0.121898
14549 time: 4:58:56.104733
Loss: 0.121898
14550 time: 4:58:57.248511
Loss: 0.121898
14551 time: 4:58:58.350692
Loss: 0.121898
14552 time: 4:58:59.434811
Loss: 0.121898
14553 time: 4:59:00.543753
Loss: 0.121898
14554 time: 4:59:01.660777
Loss: 0.121898
14555 time: 4:59:02.809153
Loss: 0.121898
14556 time: 4:59:03.919219
Loss: 0.121898
14557 time: 4:59:05.067620
Loss: 0.121898
14558 time: 4:59:06.169986
Loss: 0.121898
14559 time: 4:59:07.288749
Loss: 0

14732 time: 5:02:30.912921
Loss: 0.121898
14733 time: 5:02:32.017727
Loss: 0.121898
14734 time: 5:02:33.243319
Loss: 0.121898
14735 time: 5:02:34.519147
Loss: 0.121898
14736 time: 5:02:35.829662
Loss: 0.121898
14737 time: 5:02:37.165225
Loss: 0.121898
14738 time: 5:02:38.512116
Loss: 0.121898
14739 time: 5:02:39.832092
Loss: 0.121898
14740 time: 5:02:41.158514
Loss: 0.121898
14741 time: 5:02:42.457108
Loss: 0.121898
14742 time: 5:02:43.672312
Loss: 0.121898
14743 time: 5:02:44.895068
Loss: 0.121898
14744 time: 5:02:46.069941
Loss: 0.121898
14745 time: 5:02:47.273756
Loss: 0.121898
14746 time: 5:02:48.445902
Loss: 0.121898
14747 time: 5:02:49.635824
Loss: 0.121898
14748 time: 5:02:50.801413
Loss: 0.121898
14749 time: 5:02:51.990891
Loss: 0.121898
14750 time: 5:02:53.150416
Loss: 0.121898
14751 time: 5:02:54.317347
Loss: 0.121898
14752 time: 5:02:55.463119
Loss: 0.121898
14753 time: 5:02:56.609840
Loss: 0.121898
14754 time: 5:02:57.796321
Loss: 0.121898
14755 time: 5:02:59.040112
Loss: 0

14928 time: 5:06:23.622824
Loss: 0.121898
14929 time: 5:06:24.820840
Loss: 0.121898
14930 time: 5:06:25.959695
Loss: 0.121898
14931 time: 5:06:27.170720
Loss: 0.121898
14932 time: 5:06:28.340119
Loss: 0.121898
14933 time: 5:06:29.571766
Loss: 0.121898
14934 time: 5:06:30.721929
Loss: 0.121898
14935 time: 5:06:31.877511
Loss: 0.121898
14936 time: 5:06:33.009723
Loss: 0.121898
14937 time: 5:06:34.217677
Loss: 0.121898
14938 time: 5:06:35.389591
Loss: 0.121898
14939 time: 5:06:36.615357
Loss: 0.121898
14940 time: 5:06:37.789915
Loss: 0.121898
14941 time: 5:06:39.002598
Loss: 0.121898
14942 time: 5:06:40.190980
Loss: 0.121898
14943 time: 5:06:41.340977
Loss: 0.121898
14944 time: 5:06:42.512926
Loss: 0.121898
14945 time: 5:06:43.700689
Loss: 0.121898
14946 time: 5:06:44.910079
Loss: 0.121898
14947 time: 5:06:46.080094
Loss: 0.121898
14948 time: 5:06:47.211665
Loss: 0.121898
14949 time: 5:06:48.358302
Loss: 0.121898
14950 time: 5:06:49.486422
Loss: 0.121898
14951 time: 5:06:50.682551
Loss: 0